In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from datetime import datetime
import json

In [2]:
file = "MasterData_Wilson.xlsx"
pd_file = pd.read_excel(file)
bakery_df = pd.DataFrame(pd_file)
bakery_df.head()

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due,Unnamed: 29,Unnamed: 30
0,2016-07-25,2016-07-25 09:04:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,4.0,...,2.25,1,0,2.25,0,2.25,0.05,2.3,NaN,NaN
1,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,2,2,0,4,0,4,0.09,4.09,NaN,NaN
2,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,3.75,1,0,3.75,0,3.75,0.38,4.13,NaN,NaN
3,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,3.8,1,0,3.8,0,3.8,0.09,3.89,NaN,NaN
4,2016-07-25,2016-07-25 09:29:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,29.0,...,3.5,1,0,3.5,0,3.5,0.08,3.58,NaN,NaN


In [3]:
bakery_df.columns

Index(['date', 'time', 'year', 'MonthN', 'Month', 'WeekN', 'Day', 'WeekD', 'H',
       'Min', '30min', 'Rtime', '30time', 'H.5', 'Operation', 'Line Item',
       'Department', 'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price',
       'Quantity', 'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax',
       'Total Due', 'Unnamed: 29', 'Unnamed: 30'],
      dtype='object')

In [4]:
bakery_df['Unnamed: 29'].value_counts()

Sat    1
2      1
Name: Unnamed: 29, dtype: int64

In [5]:
bakery_df.loc[(bakery_df['Unnamed: 29'] == 'Sat') | (bakery_df['Unnamed: 29'] == 2), :]

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due,Unnamed: 29,Unnamed: 30
393033,2018-07-22,2018-07-22 11:18:00,2018.0,7.0,Jul,29.0,22.0,Sun,11.0,18.0,...,3.8,2,0,7.6,0,7.6,0.18,7.78,2,2.0
393034,2018-07-22,2018-07-22 11:18:00,2018.0,7.0,Jul,29.0,22.0,Sun,11.0,18.0,...,3.5,4,0,14,0,14,0.32,14.32,Sat,NaN


In [6]:
bakery_df['Unnamed: 30'].value_counts()

2.0    1
Name: Unnamed: 30, dtype: int64

In [7]:
bakery_df = bakery_df[['date', 'time', 'year', 'MonthN', 'Month', 'WeekN', 'Day', 'WeekD', 'H',
       'Min', '30min', 'Rtime', '30time', 'H.5', 'Operation', 'Line Item',
       'Department', 'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price',
       'Quantity', 'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax',
       'Total Due']]

In [8]:
bakery_df.head()

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due
0,2016-07-25,2016-07-25 09:04:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,4.0,...,MACCHO,1,2.25,1,0,2.25,0,2.25,0.05,2.3
1,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,00:00:00,0,2,2,0,4,0,4,0.09,4.09
2,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,CBC,0,3.75,1,0,3.75,0,3.75,0.38,4.13
3,2016-07-25,2016-07-25 09:28:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,28.0,...,CRC,0,3.8,1,0,3.8,0,3.8,0.09,3.89
4,2016-07-25,2016-07-25 09:29:00,2016.0,7.0,Jul,31.0,25.0,Mon,9.0,29.0,...,BTRA,1.4,3.5,1,0,3.5,0,3.5,0.08,3.58


In [9]:
bakery_df.dtypes

date             datetime64[ns]
time                     object
year                    float64
MonthN                  float64
Month                    object
WeekN                   float64
Day                     float64
WeekD                    object
H                       float64
Min                     float64
30min                   float64
Rtime                    object
30time                   object
H.5                     float64
Operation                object
Line Item                object
Department               object
Category                 object
Supplier                 object
Supplier Code            object
Cost                     object
Price                    object
Quantity                 object
Modifiers                object
Subtotal                 object
Discounts                object
Net Total                object
Tax                      object
Total Due                object
dtype: object

In [10]:
bakery_df['year'] = bakery_df['year'].fillna(0.0).astype(int)

In [11]:
bakery_df['year'].value_counts()

2017    203373
2018    186308
2016     79201
0            1
Name: year, dtype: int64

In [12]:
bakery_df.loc[bakery_df['year'] == 0, :]

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due
416580,2018-09-03,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0


In [13]:
bakery_df = bakery_df.drop(index=416580)
bakery_df.loc[bakery_df['year'] == 0, :]

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due


In [14]:
bakery_df['MonthN'] = bakery_df['MonthN'].fillna(0.0).astype(int)

In [15]:
bakery_df['MonthN'].value_counts()

9     54096
10    52943
11    48566
8     47183
5     39560
3     36752
6     34988
7     34876
4     33477
12    29130
2     28677
1     28634
Name: MonthN, dtype: int64

In [16]:
bakery_df['WeekN'] = bakery_df['WeekN'].fillna(0.0).astype(int)
bakery_df['WeekN'].value_counts()

46    14343
41    13682
38    13388
39    13320
42    12906
40    12576
45    12565
36    12330
44    11675
37    11619
35    11165
47    10787
19    10562
33    10542
31    10429
34    10353
43    10182
32     9902
20     9035
22     8740
24     8661
21     8426
12     8366
15     8364
23     8333
13     8238
11     8152
25     8148
14     8031
10     7991
49     7987
4      7963
17     7916
16     7768
18     7612
50     7589
5      7530
7      7529
9      7524
29     7517
30     7280
3      7137
8      6994
6      6959
28     6954
27     6876
51     6859
26     6725
52     6211
48     5696
2      5577
53     4109
1      1759
Name: WeekN, dtype: int64

In [17]:
bakery_df['Day'] = bakery_df['Day'].fillna(0.0).astype(int)
bakery_df['Day'].value_counts()

14    17191
13    16874
6     16853
12    16681
11    16531
3     16328
8     16245
17    16059
18    15769
22    15733
9     15727
7     15661
10    15440
19    15419
20    15372
28    15343
16    15308
15    15214
21    15156
5     15108
2     15001
27    14757
29    14622
26    14343
30    14253
4     14007
1     13917
23    13796
25    13739
24    13435
31     9000
Name: Day, dtype: int64

In [18]:
bakery_df.head()

,date,time,year,MonthN,Month,WeekN,Day,WeekD,H,Min,...,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due
0,2016-07-25,2016-07-25 09:04:00,2016,7,Jul,31,25,Mon,9.0,4.0,...,MACCHO,1,2.25,1,0,2.25,0,2.25,0.05,2.3
1,2016-07-25,2016-07-25 09:28:00,2016,7,Jul,31,25,Mon,9.0,28.0,...,00:00:00,0,2,2,0,4,0,4,0.09,4.09
2,2016-07-25,2016-07-25 09:28:00,2016,7,Jul,31,25,Mon,9.0,28.0,...,CBC,0,3.75,1,0,3.75,0,3.75,0.38,4.13
3,2016-07-25,2016-07-25 09:28:00,2016,7,Jul,31,25,Mon,9.0,28.0,...,CRC,0,3.8,1,0,3.8,0,3.8,0.09,3.89
4,2016-07-25,2016-07-25 09:29:00,2016,7,Jul,31,25,Mon,9.0,29.0,...,BTRA,1.4,3.5,1,0,3.5,0,3.5,0.08,3.58


In [19]:
cln_bakery_df = bakery_df[['time', 'WeekN', 'WeekD', 'Rtime', 'Operation', 'Line Item', 'Department', \
                           'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price','Quantity', \
                           'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax', 'Total Due']]

In [20]:
cln_bakery_df.head()

,time,WeekN,WeekD,Rtime,Operation,Line Item,Department,Category,Supplier,Supplier Code,Cost,Price,Quantity,Modifiers,Subtotal,Discounts,Net Total,Tax,Total Due
0,2016-07-25 09:04:00,31,Mon,09:04:00,SALE,Macaron Chocolate,Pastry,general,Cook Au Vin,MACCHO,1,2.25,1,0,2.25,0,2.25,0.05,2.3
1,2016-07-25 09:28:00,31,Mon,09:28:00,SALE,Type in,general,general,not tracked,00:00:00,0,2,2,0,4,0,4,0.09,4.09
2,2016-07-25 09:28:00,31,Mon,09:28:00,SALE,Latte,Coffee,general,Chicago Beverage Company,CBC,0,3.75,1,0,3.75,0,3.75,0.38,4.13
3,2016-07-25 09:28:00,31,Mon,09:28:00,SALE,Chocolate croissant (to go),Croissants,To Go,Cook Au Vin,CRC,0,3.8,1,0,3.8,0,3.8,0.09,3.89
4,2016-07-25 09:29:00,31,Mon,09:29:00,SALE,Baguette tradition,Bread,general,Cook Au Vin,BTRA,1.4,3.5,1,0,3.5,0,3.5,0.08,3.58


In [21]:
cln_bakery_df['Operation'].value_counts()

SALE      468394
RETURN       488
Name: Operation, dtype: int64

In [22]:
cln_bakery_df['Supplier'].value_counts()

not tracked                 238976
Cook Au Vin                 199936
Chicago Beverage Company     15187
French Feast                 13933
Viola Imports                  601
La Colombe                     248
Chef warehouse                   1
Name: Supplier, dtype: int64

In [23]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [144]:
review_list = []

In [120]:
bakery_link = "https://www.yelp.com/biz/la-boulangerie-chicago-4?sort_by=date_desc"
bakery_link_2 = "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=20&sort_by=date_desc"
bakery_link_3 = "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=40&sort_by=date_desc"
bakery_link_4 = "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=60&sort_by=date_desc"

In [157]:
bakery_response = requests.get(bakery_link_4)
bakery_soup = bs(bakery_response.text, 'html.parser')

In [158]:
bakery_results = bakery_soup.body.find_all('div', class_='review--with-sidebar')

In [159]:
for review in bakery_results:
    user_info = []
    user_stats = []
    user_name = '' 
    user_loc = ''
    user_friends = ''
    user_reviews = ''
    user_photos = ''
    review_info = []
    review_stars = ''
    review_date = ''
    review_text = ''
    user_check_ins = ''
    elite = ''
    elite_status = ''
    photo = []
    photo_links = []
    review_data = []
    
    user_information = {}
    
    try:
        user_info = review.find('ul', class_='user-passport-info')
        user_stats = review.find('ul', class_='user-passport-stats')
        review_info = review.find('div', class_='review-content')
        review_footer = review.find('div', class_='review-footer')
        
        try:
            user_name = user_info.find('li', class_='user-name').find('a').text.strip()
            user_profile_link = user_info.find('a', class_='user-display-name').attrs['href']
            user_profile_link = "https://www.yelp.com" + user_profile_link
            user_loc = user_info.find('li', class_='user-location').find('b').text.strip()
            review_stars = review_info.find('div', class_='i-stars').attrs['title']
            review_stars = int(float(review_stars.split()[0]))
            review_date = review_info.find('span', class_='rating-qualifier').text.strip()
            review_text = review_info.find('p').text
        except AttributeError as error:
            print(error)
            
        try:
            user_friends = user_stats.find('li', class_='friend-count').find('b').text.strip()
        except AttributeError as error:
            if user_stats:
                user_friends = '0'
        
        try:
            user_reviews = user_stats.find('li', class_='review-count').find('b').text.strip()
        except AttributeError as error:
            if user_stats:
                user_reviews = '0'
        
        try:
            user_photos = user_stats.find('li', class_='photo-count').find('b').text.strip()
        except AttributeError as error:
            if user_stats:
                user_photos = '0'
        
        try:
            elite = user_stats.find('li', class_='is-elite')
            if elite:
                elite_status = 'elite'
            else:
                elite_status = 'not elite'
        except AttributeError as error:
            print(error)
        
        try:
            user_check_ins = review_info.find('li', class_='review-tags_item').find('span').text.strip()
            user_check_ins = user_check_ins.split()[0]
        except AttributeError as error:
            if review_info:
                user_check_ins = '0'
        
        try:
            photos = review_info.find('ul', class_='photo-box-grid')
            if photos:
                try:
                    picture = photos.find_all('div', class_='photo-box')
                    for pic in picture:
                        pic_link = ''
                        pic_link = pic.find('img').attrs['data-async-src']
                        photo_links.append(pic_link)
                except Exception as error:
                    print(error)

        except AttributeError as error:
            if review_info:
                photo_links = []
        
        try:
            review_options = review_footer.find_all('li', class_='vote-item')
            review_data = {
                'review_type' : [],
                'review_count' : []
            }
            for option in review_options:
                review_type = ''
                review_count = ''
                review_type = option.find('a').find('span', class_='vote-type').text.strip()
                review_count = option.find('a').find('span', class_='count').text.strip()
                if review_count == '':
                    review_count = '0'
                review_data['review_type'].append(review_type)
                review_data['review_count'].append(review_count)
        except AttributeError as error:
            print(error)

        if (user_name and user_loc and user_friends and user_reviews and user_photos and review_stars \
            and review_date and user_check_ins and user_profile_link and review_text):
            try:
                user_check_ins = int(user_check_ins)
            except ValueError as error:
                user_check_ins = 0
            user_information = {
                'user_name' : user_name,
                'user_location' : user_loc,
                'user_link' : user_profile_link,
                'user_friends' : int(user_friends),
                'user_reviews' : int(user_reviews),
                'user_photos' : int(user_photos),
                'stars' : review_stars,
                'review_date' : review_date,
                'review_text' : review_text,
                'review_photos' : photo_links,
                'review_feedback' : review_data,
                'check_ins' : user_check_ins,
                'elite_status' : elite_status
            }

            review_list.append(user_information)
    except AttributeError as error:
        print(error)

print(json.dumps(review_list, indent=4, sort_keys=True))

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
[
    {
        "check_ins": 0,
        "elite_status": "elite",
        "review_date": "12/31/2018",
        "review_feedback": {
            "review_count": [
                "0",
                "1",
                "1"
            ],
            "review_type": [
                "Useful",
                "Funny",
                "Cool"
            ]
        },
        "review_photos": [],
        "review_text": "I adore this neighborhood bakery. It is a few blocks from where I live, so I find myself here on many weekend mornings. Everything is so fresh -- from the made to order sandwiches, to the freshly baked breads and pastries. The delicious smells from the kitchen and the large crowds say it all! I have only been here in the winter, but I imagine the patio would be a great place to enjoy a quick coffee and bite. The only suggestion I have is to potent

In [160]:
len(review_list)

71

In [161]:
print(review_list[0])

{'user_name': 'Courtney J.', 'user_location': 'Chicago, IL', 'user_link': 'https://www.yelp.com/user_details?userid=aqkJ_0_ILP_myoK5-r2G-Q', 'user_friends': 263, 'user_reviews': 61, 'user_photos': 10, 'stars': 5, 'review_date': '12/31/2018', 'review_text': 'I adore this neighborhood bakery. It is a few blocks from where I live, so I find myself here on many weekend mornings. Everything is so fresh -- from the made to order sandwiches, to the freshly baked breads and pastries. The delicious smells from the kitchen and the large crowds say it all! I have only been here in the winter, but I imagine the patio would be a great place to enjoy a quick coffee and bite. The only suggestion I have is to potentially have a better system for keeping track of "to-go" orders. I\'ve waited quite a while for to-go items that were forgotten by the staff who took my order.', 'review_photos': [], 'review_feedback': {'review_type': ['Useful', 'Funny', 'Cool'], 'review_count': ['0', '1', '1']}, 'check_ins'

In [168]:
for review in review_list:
    counts = review['review_feedback']['review_count']
    review['useful_rating_count'] = counts[0]
    review['funny_rating_count'] = counts[1]
    review['cool_rating_count'] = counts[2]
print(review_list[0])

{'user_name': 'Courtney J.', 'user_location': 'Chicago, IL', 'user_link': 'https://www.yelp.com/user_details?userid=aqkJ_0_ILP_myoK5-r2G-Q', 'user_friends': 263, 'user_reviews': 61, 'user_photos': 10, 'stars': 5, 'review_date': '12/31/2018', 'review_text': 'I adore this neighborhood bakery. It is a few blocks from where I live, so I find myself here on many weekend mornings. Everything is so fresh -- from the made to order sandwiches, to the freshly baked breads and pastries. The delicious smells from the kitchen and the large crowds say it all! I have only been here in the winter, but I imagine the patio would be a great place to enjoy a quick coffee and bite. The only suggestion I have is to potentially have a better system for keeping track of "to-go" orders. I\'ve waited quite a while for to-go items that were forgotten by the staff who took my order.', 'review_photos': [], 'review_feedback': {'review_type': ['Useful', 'Funny', 'Cool'], 'review_count': ['0', '1', '1']}, 'check_ins'

In [169]:
review_df = pd.DataFrame(review_list).set_index('user_name')
review_df.head()

,check_ins,cool_rating_count,elite_status,funny_rating_count,review_date,review_feedback,review_photos,review_text,stars,useful_rating_count,user_friends,user_link,user_location,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,
Courtney J.,0,1,elite,1,12/31/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[],I adore this neighborhood bakery. It is a few ...,5,0,263,https://www.yelp.com/user_details?userid=aqkJ_...,"Chicago, IL",10,61
Sabrina G.,1,0,elite,0,12/23/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[https://s3-media4.fl.yelpcdn.com/bphoto/SqCN8...,I've been dreaming of quiche and croissants as...,5,0,195,https://www.yelp.com/user_details?userid=LlGh8...,"Chicago, IL",81,50
Alfie B.,0,1,elite,0,10/10/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[https://s3-media4.fl.yelpcdn.com/bphoto/oLYVK...,We came to La Boulangerie after recommendation...,5,0,16,https://www.yelp.com/user_details?userid=mymQs...,"Columbia, SC",323,106
Anna L.,0,1,elite,1,10/9/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[],This is a great breakfast spot. Even better if...,5,1,28,https://www.yelp.com/user_details?userid=1aah9...,"Columbia, SC",145,175
Jordan E.,0,0,not elite,0,10/2/2018,"{'review_type': ['Useful', 'Funny', 'Cool'], '...",[],"Everytime i go here im like, wait a minute, is...",3,0,0,https://www.yelp.com/user_details?userid=0ANzg...,"Chicago, IL",6,20


In [170]:
review_df.columns

Index(['check_ins', 'cool_rating_count', 'elite_status', 'funny_rating_count',
       'review_date', 'review_feedback', 'review_photos', 'review_text',
       'stars', 'useful_rating_count', 'user_friends', 'user_link',
       'user_location', 'user_photos', 'user_reviews'],
      dtype='object')

In [172]:
review_df = review_df[['review_date', 'stars', 'user_location', 'review_text', 'check_ins', 'cool_rating_count', 'useful_rating_count', \
                       'funny_rating_count', 'review_photos', 'elite_status', 'user_link', 'user_friends', \
                       'user_photos', 'user_reviews']]
review_df.head()

,review_date,stars,user_location,review_text,check_ins,cool_rating_count,useful_rating_count,funny_rating_count,review_photos,elite_status,user_link,user_friends,user_photos,user_reviews
user_name,,,,,,,,,,,,,,
Courtney J.,12/31/2018,5,"Chicago, IL",I adore this neighborhood bakery. It is a few ...,0,1,0,1,[],elite,https://www.yelp.com/user_details?userid=aqkJ_...,263,10,61
Sabrina G.,12/23/2018,5,"Chicago, IL",I've been dreaming of quiche and croissants as...,1,0,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/SqCN8...,elite,https://www.yelp.com/user_details?userid=LlGh8...,195,81,50
Alfie B.,10/10/2018,5,"Columbia, SC",We came to La Boulangerie after recommendation...,0,1,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/oLYVK...,elite,https://www.yelp.com/user_details?userid=mymQs...,16,323,106
Anna L.,10/9/2018,5,"Columbia, SC",This is a great breakfast spot. Even better if...,0,1,1,1,[],elite,https://www.yelp.com/user_details?userid=1aah9...,28,145,175
Jordan E.,10/2/2018,3,"Chicago, IL","Everytime i go here im like, wait a minute, is...",0,0,0,0,[],not elite,https://www.yelp.com/user_details?userid=0ANzg...,0,6,20


In [173]:
review_df.to_csv('review_information.csv')